In [ ]:
#Source: https://scikit-learn.org/stable/modules/feature_extraction.html#vectorizing-a-large-text-corpus-with-the-hashing-trick
# Source: https://stackoverflow.com/questions/17536394/how-can-i-reduce-memory-usage-of-scikit-learn-vectorizers
# Reasoning is that TFIDF Vectorizer create very sparse matrix, which resulted in a 284GB matrix that needed to be held in memory in order to process it.
# Initialize HashingVectorizer with a reasonable number of features
hash_vectorizer = HashingVectorizer(n_features=2**10, alternate_sign=False, norm=None)

# Transform the text data into hashed feature space using the string format
hashed_matrix = hash_vectorizer.transform(filtered_df['filtered_text_str'])

# Apply TfidfTransformer to add IDF weighting
tfidf_transformer = TfidfTransformer()
tfidf_matrix = tfidf_transformer.fit_transform(hashed_matrix)

# Check the shape and sparsity of the transformed TF-IDF matrix
print(f"Shape of tfidf_matrix: {tfidf_matrix.shape}")
print(f"Number of non-zero elements: {tfidf_matrix.nnz}")
print(f"Sparsity: {(1 - (tfidf_matrix.nnz / (tfidf_matrix.shape[0] * tfidf_matrix.shape[1]))) * 100:.2f}%")


In [ ]:
# Compute cosine similarity matrix, which measures the similarity between documents
# Source: https://medium.com/@anurag-jain/tf-idf-vectorization-with-cosine-similarity-eca3386d4423
similarity_matrix = cosine_similarity(tfidf_matrix)


In [ ]:
# Use TfidfVectorizer with min_df and ngram_range to reduce dimensionality
vectorizer = TfidfVectorizer(
    lowercase = False,  # Do not convert to lowercase,because it is already converted
    min_df = 2,  # Ignore terms that appear in less than 2 documents
    ngram_range=(1, 2),  # Consider both unigrams and bigrams
    max_features=10000,  # Further limit the size of the vocabulary
)

# Fit and transform the text data
#tfidf_matrix = vectorizer.fit_transform(filtered_df['filtered_text_str'])

In [ ]:
vectorizer = TfidfVectorizer(
    max_df=0.5,
    min_df=5,
    max_features=10,
    stop_words="english",
)
# Fit and transform the text data
tfidf_matrix = vectorizer.fit_transform(filtered_df['filtered_text_str'])

In [ ]:

# Train a simple logistic regression model for demonstration
clf = LogisticRegression(max_iter=1000)
clf.fit(tfidf_matrix, filtered_df['target'])

# Function to print top informative features for binary classification
def print_top_features(vectorizer, clf, top_n=20):
    """Prints the top n informative features for each class using model coefficients."""
    feature_names = vectorizer.get_feature_names_out()
    coef = clf.coef_[0]  # Use the first (and only) set of coefficients for binary classification
    
    # Top features with the most negative coefficients (indicative of Class 0)
    top_features_class_0 = np.argsort(coef)[:top_n]
    print("\nTop 20 Words Indicative of Class 0 (Stock Price Down):")
    print(" ".join(feature_names[j] for j in top_features_class_0))
    
    # Top features with the most positive coefficients (indicative of Class 1)
    top_features_class_1 = np.argsort(coef)[-top_n:]
    print("\nTop 20 Words Indicative of Class 1 (Stock Price Up):")
    print(" ".join(feature_names[j] for j in top_features_class_1))
    
    return top_features_class_0, top_features_class_1

# Print the most informative features
top_features_class_0, top_features_class_1 = print_top_features(vectorizer, clf)

In [ ]:

# Plotting the most indicative words for Class 0
plt.figure(figsize=(12, 6))
plt.barh([vectorizer.get_feature_names_out()[i] for i in top_features_class_0], clf.coef_[0][top_features_class_0], color='blue')
plt.title('Top 20 Words Indicative of Class 0 (Stock Price Down)')
plt.xlabel('Coefficient Value')
plt.gca().invert_yaxis()
plt.show()

In [ ]:

# Plotting the most indicative words for Class 1
plt.figure(figsize=(12, 6))
plt.barh([vectorizer.get_feature_names_out()[i] for i in top_features_class_1], clf.coef_[0][top_features_class_1], color='green')
plt.title('Top 20 Words Indicative of Class 1 (Stock Price Up)')
plt.xlabel('Coefficient Value')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# Check the shape of the tfidf_matrix
print(f"Shape of tfidf_matrix: {tfidf_matrix.shape}")

# Number of documents (rows) and terms (columns)
num_documents, num_features = tfidf_matrix.shape
print(f"Number of documents: {num_documents}")
print(f"Number of features (terms): {num_features}")

# Check the number of non-zero elements
non_zero_elements = tfidf_matrix.nnz
print(f"Number of non-zero elements: {non_zero_elements}")

# Calculate the total number of elements
total_elements = num_documents * num_features
print(f"Total number of elements: {total_elements}")

# Calculate the sparsity of the matrix
sparsity = (1 - (non_zero_elements / total_elements)) * 100
print(f"Sparsity of the tfidf_matrix: {sparsity:.2f}%")


In [ ]:
# Compute cosine similarity matrix, which measures the similarity between documents
# Source: https://medium.com/@anurag-jain/tf-idf-vectorization-with-cosine-similarity-eca3386d4423
similarity_matrix = cosine_similarity(tfidf_matrix)

# Convert similarity matrix to a DataFrame for easier handling
similarity_df = pd.DataFrame(similarity_matrix)

In [ ]:
# Function to compute cosine similarity in batches
def compute_similarity_in_batches(tfidf_matrix, batch_size=1000):
    num_docs = tfidf_matrix.shape[0]
    similarity_results = []
    
    # Process the similarity computation in batches
    for start in range(0, num_docs, batch_size):
        end = min(start + batch_size, num_docs)
        print(f"Processing batch: {start} to {end}")
        
        # Compute similarities for the batch
        batch_similarity = cosine_similarity(tfidf_matrix[start:end], tfidf_matrix)
        
        # Convert to a sparse matrix to save memory (optional)
        batch_similarity_sparse = pd.DataFrame(batch_similarity)
        
        # Append the results
        similarity_results.append(batch_similarity_sparse)
    
    # Combine all batches into one DataFrame
    return pd.concat(similarity_results, axis=0)

# Using the function to compute the similarity matrix in smaller chunks
similarity_df = compute_similarity_in_batches(tfidf_matrix, batch_size=500)

# Display the shape to confirm the complete matrix
print(f"Computed similarity matrix shape: {similarity_df.shape}")


In [ ]:
# Find indices of the most and least similar documents (excluding self-similarity)
np.fill_diagonal(similarity_matrix, 0)
most_similar_indices = np.unravel_index(np.argmax(similarity_matrix, axis=None), similarity_matrix.shape)
least_similar_indices = np.unravel_index(np.argmin(similarity_matrix, axis=None), similarity_matrix.shape)

In [ ]:

# Inspect the most similar documents
similar_doc_1 = news_df.iloc[most_similar_indices[0]]
similar_doc_2 = news_df.iloc[most_similar_indices[1]]

print("\nMost Similar Documents:\n")
print("Document 1:")
print(similar_doc_1['content'])
print("\nDocument 2:")
print(similar_doc_2['content'])

In [ ]:

# Inspect the least similar documents
dissimilar_doc_1 = news_df.iloc[least_similar_indices[0]]
dissimilar_doc_2 = news_df.iloc[least_similar_indices[1]]

print("\n\nMost Dissimilar Documents:\n")
print("Document 1:")
print(dissimilar_doc_1['content'])
print("\nDocument 2:")
print(dissimilar_doc_2['content'])

